In [24]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import datetime
#%load_ext tensorboard


In [22]:
def create_train_validate_gen(model_type, rescaling_factor, image_path, batch_size):
    # Create train and validate generators for bacteria data from ImageDataGenerator 

    if 'resnet' in model_type :
        train_datagen = ImageDataGenerator(rescale = rescaling_factor,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,                   
                    validation_split = 0.2,
                    preprocessing_function = preprocess_input)  # Use specific data preprocessing for resnet
                
    else:
        
        train_datagen = ImageDataGenerator(rescale = rescaling_factor,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,                   
                    validation_split = 0.2,)      
        
    # Train and validate generators
    train_generator = train_datagen.flow_from_directory(
                  image_path,
                  target_size = (224,224),
                  batch_size = batch_size,
                  class_mode = 'categorical',
                  subset = 'training')
    
    validation_generator = train_datagen.flow_from_directory(
                       image_path, 
                       target_size = (224, 224),
                       batch_size= batch_size,
                       class_mode='categorical',
                       subset='validation')    # set as validation data)
    
    
    return train_datagen, train_generator, validation_generator
    

In [15]:
def create_model_head(baseModel, dense1, dropout1, num_classes):
    # Create model head to put atop pre-trained networks
    
    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(33, activation="softmax")(headModel)
    
    return headModel
    

In [17]:
def train_head_model(init_lr, batch_size, num_epochs, train_generator, validation_generator, callbacks):
    # Compile and train the head model component
    
    print("[INFO] compiling model...")
    opt = Adam(lr = init_lr, decay = init_lr / num_epochs)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
    

    print("[INFO] training head...")
    model.fit(train_generator,
              steps_per_epoch = train_generator.samples // batch_size,
              validation_data = validation_generator, 
              validation_steps = validation_generator.samples // batch_size,
              epochs = num_epochs,
              callbacks=[tensorboard_callback])
    

 # VGG16 Model:

In [ ]:
from tensorflow.keras.applications import VGG16

# VGG16 Model Parameters 
model_type = 'vgg16'
num_epochs = 35 
batch_size = 32
init_lr = 1e-3
rescaling_factor = 1./255
image_path = './Bacteria/Train'

# Data generators
train_datagen, train_generator, validation_generator = create_train_validate_gen(model_type, rescaling_factor, image_path, batch_size) 

# Create base model
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze kayers of base model
for layer in baseModel.layers:
    layer.trainable = False

# Create head model
headModel = create_model_head(baseModel, 512, 0.2, 33)

# Place the head FC model on top of the base model 
model = Model(inputs = baseModel.input, outputs = headModel)
   
# Create log directory for train and validate and callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

# Train model head
train_head_model(init_lr, batch_size, num_epochs, train_generator, validation_generator, tensorboard_callback)    
   

# Save model    
model.save('/models/Vgg16_model') 

Found 514 images belonging to 33 classes.
Found 109 images belonging to 33 classes.
[INFO] compiling model...
[INFO] training head...
Epoch 1/35
 1/16 [>.............................] - ETA: 0s - loss: 3.5392 - accuracy: 0.0625WARNING:tensorflow:From C:\Users\Ed\.conda\envs\tf\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
16/16 [==============================] - 159s 10s/step - loss: 4.7971 - accuracy: 0.1211 - val_loss: 2.3992 - val_accuracy: 0.3125
Epoch 2/35
16/16 [==============================] - 165s 10s/step - loss: 2.3798 - accuracy: 0.3008 - val_loss: 1.7486 - val_accuracy: 0.4688
Epoch 3/35
 7/16 [============>.................] - ETA: 58s - loss: 1.9594 - accuracy: 0.3557 

# ResNet50 Model:

In [47]:
from tensorflow.keras.applications.resnet50 import ResNet50

# ResNet50 Model Parameters 
model_type = 'resnet50'
num_epochs = 50 
batch_size = 32
init_lr = 1e-3
rescaling_factor = None
image_path = './Bacteria/Train'

# Data generators
train_datagen, train_generator, validation_generator = create_train_validate_gen(model_type, rescaling_factor, image_path, batch_size) 

# Create base model
baseModel = ResNet50(include_top=False, weights = 'imagenet', input_tensor=Input(shape=(224, 224, 3)), pooling = 'avg')

# Freeze kayers of base model
for layer in baseModel.layers:
    layer.trainable = False

# Create head model
headModel = create_model_head(baseModel, 1024, 0.2, 33)

# Place the head FC model on top of the base model 
model = Model(inputs = baseModel.input, outputs = headModel)
   
# Create log directory for train and validate and callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

# Train model head
train_head_model(init_lr, batch_size, num_epochs, train_generator, validation_generator, tensorboard_callback)    
   

# Save model    
model.save('/models/ResNet50_model') 

Found 514 images belonging to 33 classes.
Found 109 images belonging to 33 classes.


In [ ]:
# Tensor board

%tensorboard --logdir=logs/fit --host localhost --port 8088